# Tutorial overview

## Structural manipulation and preparation

The `OgStructure` class enables you to do various structure manuipulation operations:

- You can adsorb an atom on a surface by simply calling the `add_atom_to_surface(atom)`:

In [ ]:
from oganesson.ogstructure import OgStructure

og=OgStructure(file_name='examples/structures/MoS2.vasp')
og.add_atom_to_surface('Li').structure.to('MoS2_Li.vasp','poscar')

- You can scan for multiple adsorption positions on a surface by using the `adsorption_scanner(atom)` method:

In [ ]:
from oganesson.ogstructure import OgStructure
from ase.build import surface
s = surface('Au', (1, 1, 1), 9)
s.center(vacuum=10, axis=2)

og=OgStructure(s)
og.add_atom_to_surface('O').structure.to('Au_O.cif','poscar')

og=OgStructure(s)
ad_structures = og.adsorption_scanner('O')
counter = 0
for s in ad_structures:
    counter += 1
    s().to(str(counter) + '.cif','poscar')

- You can add an interstitial defect to a structure by calling the `add_interstitial(atom)`:

In [ ]:
s = OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif')
s.add_interstitial('H')().to('H_interstitial.cif','cif')

- You can generate the entire VASP input POSCARs for the NEB diffusion pathway of an atom in the structure:

In [ ]:
from oganesson.ogstructure import OgStructure
og = OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif')
og.generate_neb('Li', r=3)


- To generate an alloy supercell with multiple components based on a template supercell, you can use the `substitutions_random()` method to randomly replace the atom in the supercell with other atoms at specific proportions.

In [ ]:
from oganesson.genetic_algorithms import GA
from oganesson.ogstructure import OgStructure
from ase.build import bulk

Cu = bulk('Cu', 'fcc', a=3.6)
structure = OgStructure(Cu.repeat([4,4,4]))
structure.substitutions_random('Cu',{'Al':16, 'Cr':16, 'Ti':16, 'V':16})
structure()

- To extract dynamics data from VASP OUTCAR files, there is an option to rely only on the available `OUTCAR` file, with the assumption that the first line of the `POSCAR` in the calculation had the correct assignment of species, and that there was not too many of them. However, it is better to obtain the species from the `POSCAR` file used in the calculation. So typically, one should provide both files to the `Outcar` class using the `poscar_file` parameter in the constructor.

In [ ]:
from oganesson.io.vasp import Outcar

outcar = Outcar('examples/')
outcar.write_md_data()

## Calculation of the structural properties

- You can obtain the radial distribution fuction (RDF) using the `rdf()` function, which is borrowed from the ASE library.

In [ ]:
from oganesson.ogstructure import OgStructure
from ase.build import bulk

Cu = bulk('Cu', 'fcc', a=3.6)
structure = OgStructure(Cu.repeat([4,4,4]))
structure.substitutions_random('Cu',{'Al':16, 'Cr':16, 'Ti':16, 'V':16})
print(structure.get_rdf(rmax=4,nbins=100,elements=[13,13])[0])

- You can plot the simulated XRD for your structure using the `xrd()` function.

In [ ]:
from oganesson.ogstructure import OgStructure
from ase.build import bulk

Cu = bulk('Cu', 'fcc', a=3.6)
structure = OgStructure(Cu.repeat([4,4,4]))
structure.substitutions_random('Cu',{'Al':16, 'Cr':16, 'Ti':16, 'V':16})
structure.xrd()

# Machine learning

## Machine learning descriptors

In [ ]:
from oganesson.descriptors import BACD, SymmetryFunctions, DScribeSOAP, DScribeSineMatrix
from oganesson.ogstructure import OgStructure


bacd = BACD(OgStructure(file_name='examples/structures/mp-541001_LiInI4.cif'))
bacd.describe()

sf = DScribeSineMatrix(OgStructure(file_name='examples/structures/mp-541001_LiInI4.cif'))
sf.describe()


In [ ]:
from oganesson.descriptors import BACD, Describe
# from oganesson.descriptors import DScribeSOAP, DscribeACSF
from oganesson.ogstructure import OgStructure

bacd = BACD(OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif'))
print(bacd.describe())
# soap = DScribeSOAP(OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif'))
# acsf = DscribeACSF(OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif'))

print(Describe.describe(OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif'),BACD))

## Generation of machine learning datasets

In [ ]:
import requests
from oganesson.descriptors import Describe, BACD
from oganesson.ogstructure import OgStructure
from pymatgen.core.structure import Structure
import pandas as pd

headers = {
    'accept': 'application/json',
    'X-API-KEY': 'iGRUQOIQAcPMw00QWQKIEegfhF8O7Gmm'
}
materials_summary = requests.get('https://api.materialsproject.org/materials/summary/?formula=ABO3&deprecated=false&_per_page=1000&_skip=0&_limit=1000&_all_fields=true&is_stable=true', headers=headers)
materials_summary = materials_summary.json()['data']
datasets = {'material_ids':[],'structures':[],'bacd':[],'formation_energy_per_atom':[]}
for material in materials_summary:
    structure = OgStructure(Structure.from_dict(material['structure']))
    datasets['material_ids'] += [material['material_id']]
    datasets['structures'] += [structure]
    datasets['bacd'] += [Describe.describe(structure,descriptor=BACD)]
    datasets['formation_energy_per_atom'] += [material['formation_energy_per_atom']]


In [ ]:

from joltml import Experiment, Xgboost, Sklearn

training_set = pd.DataFrame(datasets['bacd'])
experiment = Experiment(training_set.iloc[:800])
y = experiment.add_models([Xgboost()]).regression(targets=pd.DataFrame(datasets['formation_energy_per_atom'][:800]), splits=[0.8,0.2]).predict(training_set.iloc[:800])

In [ ]:
from oganesson.descriptors import BACD
from oganesson.ogstructure import OgStructure
bacd = BACD(OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif'))
print(bacd.is_invariant())

# Performing AIMD simulations

Here we apply a lithium-rich material, Li3PO4, to a temperature of 1000 K, and then calculate the diffusivities of the three elements.

In [ ]:
from oganesson.ogstructure import OgStructure
structure = OgStructure(file_name='examples/structures/LGPS_ChemMater_2018_30_4995_Opt.cif')
structure.simulate(temperature=1000,steps=1000,loginterval=1)
coeffs = structure.calculate_diffusivity()
print('Diffusion coefficients:', coeffs)


# Surface science

In [ ]:
from ase.build import fcc111, add_adsorbate
from ase import Atoms
from ase.constraints import FixAtoms
from ase.build import molecule
from oganesson.ogstructure import OgStructure
CO = molecule('CO')
E_CO = -14.77576957

Pt = fcc111('Pt', (2, 2, 6),periodic=True)
Pt.write('Pt.cif')
# fixed = list(range(len(Pt) - 2*4))
# Pt.constraints = [FixAtoms(indices=fixed)]
ogPt = OgStructure(Pt)
ogPt.relax()
E_Pt = ogPt.total_energy

add_adsorbate(Pt, CO, height=1.0, position='fcc',mol_index=0)
Pt.center(vacuum=10.0, axis=2)
Pt.write('Pt_CO.cif')
# fixed = list(range(len(Pt) - len(CO) - 2*4))
# Pt.constraints = [FixAtoms(indices=fixed)]
ogPt = OgStructure(Pt)
ogPt.relax()
ogPt.structure.to('Pt_CO_relaxed.cif','cif')
E_Pt_CO = ogPt.total_energy

# Find the binding energy
print('CO binding energy =',E_Pt_CO - E_Pt - E_CO)
print(E_Pt_CO, E_Pt, E_CO)

# Genetic algorithms

## Creating a population of materials, and performing evolutions

In [ ]:
from oganesson.genetic_algorithms import GA
ga = GA(species=['Na']*4 + ['H']*4, rmax=20, population_size=10)
for i in range(10):
    ga.evolve(num_offsprings=5)

## Evolving based on a given population

In [ ]:
from oganesson.genetic_algorithms import GA
from oganesson.ogstructure import OgStructure
from ase.build import bulk

Cu = bulk('Cu', 'fcc', a=3.6)
structure = OgStructure(Cu.repeat([2,2,2]))
print(structure())
ga = GA(population=structure.substitutions('Cu',{'Fe':4,'Cu':4}))
for i in range(10):
    ga.evolve()